In [36]:
import os
import numpy as np
import pickle as pkl
import random
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPool2D
from tensorflow.keras.models import Sequential
from numpy.linalg import norm



In [37]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False
model = Sequential([base_model, GlobalMaxPool2D()])

In [38]:
# Define clothing categories
categories = ["bottomwear", "topwear", "shoes", "accessories"]

# Function to extract features from images
def extract_features_from_images(image_path, model):
    try:
        img = image.load_img(image_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_expand_dim = np.expand_dims(img_array, axis=0)
        img_preprocess = preprocess_input(img_expand_dim)
        result = model.predict(img_preprocess, verbose=0).flatten()
        norm_result = result / norm(result)
        return norm_result
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return np.zeros(2048)  # Ensure consistent feature vector size


In [39]:
def prepare_dataset(base_path, model):
    labels = []
    features = []
    filenames = []

    for category in categories:
        folder_path = os.path.join(base_path, category)
        if not os.path.exists(folder_path):
            print(f"Warning: Folder {folder_path} does not exist.")
            continue
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            try:
                feature = extract_features_from_images(file_path, model)
                if feature is not None:
                    features.append(feature)
                    labels.append(category)
                    filenames.append(file_path)
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")

    return features, labels, filenames

# Specify dataset path
base_path = "C:/Users/asus/Downloads/Styling suggestions/"

# Extract features, labels, and filenames
features, labels, filenames = prepare_dataset(base_path, model)

# Save extracted data
pkl.dump(features, open("Images_features.pkl", "wb"))
pkl.dump(labels, open("labels.pkl", "wb"))
pkl.dump(filenames, open("filenames.pkl", "wb"))


In [40]:
def train_classifier(features, labels):
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)
    X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, random_state=42)
    svm_classifier = SVC(probability=True, kernel='linear')
    svm_classifier.fit(X_train, y_train)
    print("Model training completed.")
    return svm_classifier, label_encoder

In [41]:
svm_classifier, label_encoder = train_classifier(features, labels)

# Save the trained model and encoder
pkl.dump(svm_classifier, open("category_classifier.pkl", "wb"))
pkl.dump(label_encoder, open("label_encoder.pkl", "wb"))

Model training completed.


In [42]:
# Load pre-trained classifier and label encoder
svm_classifier = pkl.load(open("category_classifier.pkl", "rb"))
label_encoder = pkl.load(open("label_encoder.pkl", "rb"))

# Load extracted dataset features and filenames
database_features = pkl.load(open("Images_features.pkl", "rb"))
database_filenames = pkl.load(open("filenames.pkl", "rb"))


In [44]:
def classify_and_recommend(image_path, model, classifier, encoder, db_features, db_filenames):
    features = extract_features_from_images(image_path, model)
    category_idx = classifier.predict([features])[0]
    category = encoder.inverse_transform([category_idx])[0]
    print(f"Classified as: {category}")

    other_categories = [cat for cat in encoder.classes_ if cat != category]
    recommendations = {}

    for other_category in other_categories:
        indices = [i for i, fname in enumerate(db_filenames) if other_category in fname]
        if not indices:
            print(f"No images found for category: {other_category}")
            continue
        random_indices = random.sample(indices, min(3, len(indices)))
        recommendations[other_category] = [db_filenames[i] for i in random_indices]

    return category, recommendations

# Upload and classify an image
uploaded_image = r"uploads/OIP.jpg"
category, recommendations = classify_and_recommend(
    uploaded_image, model, svm_classifier, label_encoder, database_features, database_filenames
)

# Display results
print(f"Category: {category}")
print("Recommendations:")
for rec_category, rec_files in recommendations.items():
    print(f"{rec_category}: {rec_files}")


Error processing image uploads/OIP.jpg: [Errno 2] No such file or directory: 'uploads/OIP.jpg'
Classified as: accessories
Category: accessories
Recommendations:
bottomwear: ['C:/Users/asus/Downloads/Styling suggestions/bottomwear\\c6bc821e-631a-4221-9d92-7c7685edebd0.jpg', 'C:/Users/asus/Downloads/Styling suggestions/bottomwear\\56fda6ba-1ee9-4f97-a0fd-55f01f1e2512.jpg', 'C:/Users/asus/Downloads/Styling suggestions/bottomwear\\cbf45350-1a65-4cec-abad-0f64a4a459eb.jpg']
shoes: ['C:/Users/asus/Downloads/Styling suggestions/shoes\\f5ed50b6-79b3-4d9d-84a9-8010f5e1c581.jpg', 'C:/Users/asus/Downloads/Styling suggestions/shoes\\5d147b66-5238-4ed0-9c6f-c68286fea4ad.jpg', 'C:/Users/asus/Downloads/Styling suggestions/shoes\\da73c56d-32ac-4d66-bdb2-82c50adcdde7.jpg']
topwear: ['C:/Users/asus/Downloads/Styling suggestions/topwear\\ffa2be27-0798-488d-b9de-254de2226667.jpg', 'C:/Users/asus/Downloads/Styling suggestions/topwear\\e917e5f9-2394-4e18-a69d-bacaa9a6a3a1.jpg', 'C:/Users/asus/Downloads/Styl